In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, AutoModel
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
from transformers.integrations import TensorBoardCallback

In [2]:
def preprocess_function(examples):
    result = tokenizer(examples['dialog'], truncation=True, padding=True)
    result['label'] = examples['act']
    return result


dataset = DatasetDict.load_from_disk('/home/pavel/work/active_learning_project/exploded_dataset')

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

tokenized_dataset = dataset.map(preprocess_function)

f1 = load_metric('f1')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/87170 [00:00<?, ?ex/s]

  0%|          | 0/7740 [00:00<?, ?ex/s]

  0%|          | 0/8069 [00:00<?, ?ex/s]

In [4]:
len(dataset['train'])

87170

87170

In [68]:
model = model.from_pretrained('/home/pavel/work/active_learning_project/models/first.model')

In [69]:
validation = dataset['validation']

In [70]:
valid_dataset = validation.map(preprocess_function)

Loading cached processed dataset at /home/pavel/work/active_learning_project/exploded_dataset/validation/cache-b357b5aa22d8e053.arrow


In [71]:
import numpy as np
import torch

In [79]:
for i, a in enumerate(validation):
    # print(a)
    model_inputs=tokenizer(a['dialog'], truncation=True, padding=True, return_tensors='pt')
    # model_inputs={'input_ids':a['input_ids'], 'token_type_ids':a['token_type_ids'], 'attention_mask':a['attention_mask']}
    model_predictions = model(**model_inputs)
    # print(np.argmax(model_predictions), a['act'])
    # print(int(torch.argmax(model_predictions[0])))
    pred=torch.argmax(model_predictions[0])
    # print(model_predictions)
    # print(a['act'])
    f1.add(prediction=pred, reference=a['act'])
    # if i==30:
    #     break

In [78]:
f1.compute(average='weighted')

{'f1': 0.9032258064516129}

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_data_dir',
 '_check_all_processes_locks',
 '_check_rendez_vous',
 '_compute',
 '_create_cache_file',
 '_data_dir_root',
 '_download_and_prepare',
 '_finalize',
 '_get_all_cache_files',
 '_info',
 '_init_writer',
 '_metric_info',
 'add',
 'add_batch',
 'buf_writer',
 'cache_file_name',
 'citation',
 'codebase_urls',
 'compute',
 'config_name',
 'data',
 'data_dir',
 'description',
 'download_and_prepare',
 'experiment_id',
 'features',
 'file_paths',
 'filelock',
 'filelocks',
 'format',
 'homepage',
 'info',
 'inputs_description',
 'keep_in_memory',
 'license',
 'max_concurrent_cache_files'